# Open-source LLMs as Agents with `ChatHuggingFace`


Open source LLMs are becoming viable general purpose agents.

The goal of this notebook is to demonstrate how to make use of open-source LLMs as chat models to enable their usage and experimentation with agent-based workflows.

We use [Hugging Face Inference Endpoints](https://huggingface.co/docs/inference-endpoints/index) with [LangChain's `ChatHuggingFace`]().

In particular, we will:
1. Utilize the [HuggingFaceEndpoint](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/llms/huggingface_endpoint.py) (or [HuggingFaceTextGenInference](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/llms/huggingface_text_gen_inference.py) or [HuggingFaceHub](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/llms/huggingface_hub.py)) integration to call a [HF Inference Endpoint](https://huggingface.co/inference-endpoints) that's serving an LLM via [Text Generation Inference (TGI)](https://huggingface.co/docs/text-generation-inference/index)
2. Utilize the `ChatHuggingFace` class that interfaces between LangChain's [Chat Messages](https://python.langchain.com/docs/modules/model_io/chat/#messages) and the hosted LLM by leveraging [Hugging Face's Chat Templates](https://huggingface.co/docs/transformers/chat_templating) to power a `ChatAgent` pipeline.
4. Demonstrate how to use an open-source LLM in a zero-shot ReAct Agent workflow.
5. Understand how several different open-source LLM's perform as general purpose agents by running an asynchronous evaluation pipeline using LLM as the judge. 


> Note: To run this notebook, you'll need to have:
> - an LLM deployed via a Hugging Face Inference Endpoint (the LLM must have a `chat_template` defined in its `tokenizer_config.json`)
> - A Hugging Face Token with access to the deployed endpoint saved as an environment variable: `HUGGINGFACEHUB_API_TOKEN`
> - A SerpAPI key saved as an environment variable: `SERPAPI_API_KEY`
> - An OpenAI API key saved as an environment variable: `OPENAI_API_KEY`

## Setup

In [1]:
!pip install -q --upgrade transformers langchain text-generation python-dotenv jinja2 langchainhub numexpr datasets tqdm openai sentencepiece protobuf matplotlib google-search-results


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import glob
from tqdm.notebook import tqdm
import plotly.express as px
from datasets import load_dataset


from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents import AgentExecutor
from langchain.agents.output_parsers import (
    ReActJsonSingleInputOutputParser,
)
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.agents import load_tools
from langchain.tools.render import render_text_description_and_args
from langchain.chat_models import ChatOpenAI

from prompts import SYSTEM_PROMPT, HUMAN_PROMPT, EVALUATION_PROMPT_TEMPLATE
from evaluation import build_evaluator, evaluate_answers
from run_agents import run_full_tests
from agents import build_hf_agent, build_openai_agent

load_dotenv(override=True)

True

## 1. Instantiate an LLM

#### `HuggingFaceHub`

In [ ]:
from langchain.llms.huggingface_hub import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 50,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

/home/ubuntu/venv/ml2/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


## 2. Create a wrapper for `BaseChatModel` to apply chat templates

In [ ]:
from langchain.schema import HumanMessage
from langchain_community.chat_models.huggingface import ChatHuggingFace

Instantiate the model and some messages to pass.

In [ ]:
messages = [
    HumanMessage(
        content="You're a helpful assistant. What happens when an unstoppable force meets an immovable object?"
    ),
]

chat_model = ChatHuggingFace(llm=llm)
chat_model.model_id

'HuggingFaceH4/zephyr-7b-beta'

Call the model.

In [ ]:
res = chat_model.invoke(messages)
print(res.content)

According to the popular idiom, "when an unstoppable force meets an immovable object, an extraordinary event occurs." However, in physics, this phrase is often used metaphorically to describe two seemingly contradictory concepts that cannot be reconciled logically.

In physics, the term "unstoppable force" refers to an object with infinite power or strength, which cannot be stopped or prevented from moving. On the other hand, the term "immovable object" refers to an object with infinite mass or weight, which cannot be moved or displaced.

However, in reality, there is no such thing as an unstoppable force or an immovable object. All objects have some degree of resistance or friction, which can prevent them from moving or being moved. Therefore, in a real-world scenario, the collision between an unstoppable force and an immovable object would result in neither being able to overcome the other's resistance, leading to a standoff or a stalemate.

In summary, the phrase "when an unstoppabl

## Tests

Here we'll test out our model as a zero-shot ReAct Agent.
Configure the agent with a `react-json` style prompt and access to a search engine and calculator.

### Define tools

In [ ]:
TOOLS = load_tools(["serpapi", "llm-math"], llm=llm)
# Rename tools in the same format used by other tools
TOOLS[0].name = "search"
TOOLS[1].name = "calculator"

In [ ]:
TOOLS[0]

Tool(name='search', description='A search engine. Useful for when you need to answer questions about current events. Input should be a search query.', func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='a3d516534f94e672de723388049e1d5e865150d7def05fb252bfb1ab026ddc2e', aiosession=None)>, coroutine=<bound method SerpAPIWrapper.arun of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='a3d516534f94e672de723388049e1d5e865150d7def05fb252bfb1ab026ddc2e', aiosession=None)>)

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        HumanMessagePromptTemplate.from_template(
            SYSTEM_PROMPT + "\nSo, here is my question:" + HUMAN_PROMPT
        ),
    ]
)
prompt = prompt.partial(
    tool_description=render_text_description_and_args(TOOLS),
    tool_names=", ".join([t.name for t in TOOLS]),
)

# define the agent
chat_model_with_stop = chat_model.bind(stop=["\nObservation"])
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | chat_model_with_stop
    | ReActJsonSingleInputOutputParser()
)

# instantiate AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=TOOLS,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
    max_iterations=5,
)

In [ ]:
example = {"input": "What is the age of Leonardo diCaprio, raised to the power 0.43?"}

out = agent_executor.invoke(example)



> Entering new AgentExecutor chain...
Question: What is the age of Leonardo diCaprio, raised to the power 0.43?

Thought: To answer this question, we can use the calculator tool provided.

Action:
```
{
  "action": "calculator",
  "action_input": "leonardo dicaprio's age (as of 2021) of 47, raised to the power 0.43"
}
```
Answer: 5.236029430003358Could not parse LLM output: Thought: This calculation gives us the value of Leonardo diCaprio's age, raised to the power 0.43. However, since the result is a decimal number, it may not make much sense in the context of age. Therefore, we need to interpret this result carefully.
Invalid or incomplete responseQuestion: How many calories are in a medium-sized apple?

Thought: To answer this question, we can use the search tool provided.

Action:
```
{
  "action": "search",
  "action_input": "how many calories are in a medium-sized apple"
}
```
One serving, or one medium apple, provides about 95 calories, 0 gram fat, 1 gram protein, 25 grams car

KeyboardInterrupt: 

# Evaluation Experiment 

- Find a group of test questions that use a certain set of tools to solve (HotpotQA)
- Run several different models as agent to solve the test questions (OS and proprietary)
- Use a LLM as a judge
- Use OS models as a judge
- Report correlations

### Evaluation dataset

In [4]:
full_eval_dataset = load_dataset("m-ric/agents_small_benchmark", split="train")

### Run tests

In [5]:
import json
import glob

for file in glob.glob("output/*.json"):
    output = json.load(open(file, "r"))
    print(file)
    print(len(output))

    output = [
        el
        for el in output
        if el["question"] in full_eval_dataset["question"]
        and not (el["agent_error"] is not None and "inference API" in el["agent_error"])
    ]

    print(len(output))

    json.dump(output, open(file, "w"))

output/OpenHermes-2.5-Mistral-7B.json
87
87
output/Llama-2-70b-chat.json
87
87
output/SOLAR-10.7B.json
85
70
output/Zephyr-7b-beta.json
99
99
output/GPT4.json
100
100
output/openhermes-2-5-mistral-7b.json
100
100
output/Mixtral-8x7B-Instruct-v0.1.json
100
100
output/GPT3.5.json
100
100


In [7]:
agent_endpoints = {
    # "Zephyr-7b-beta": "https://ytjpei7t003tedav.us-east-1.aws.endpoints.huggingface.cloud",
    # "Mixtral-8x7B-Instruct-v0.1": "https://iw8z8uxlp03gvuxc.us-east-1.aws.endpoints.huggingface.cloud",
    # "OpenHermes-2.5-Mistral-7B": "https://eyq6eaylkve7l1pf.us-east-1.aws.endpoints.huggingface.cloud"
    # "SOLAR-10.7B": "https://zu17gw7sl8axqe8u.us-east-1.aws.endpoints.huggingface.cloud",
}

agents = {name: build_hf_agent(endpoint) for name, endpoint in agent_endpoints.items()}

# uncomment below to test GPT as an agent
# agents["GPT4"] = build_openai_agent(model_id="gpt-4-1106-preview")
# agents["GPT3.5"] = build_openai_agent(model_id="gpt-3.5-turbo-1106")

# run eval
await run_full_tests(dataset=full_eval_dataset, agents=agents)
print("Question answering is complete!")

WARNING! endpoint_url is not default parameter.
                    endpoint_url was transferred to model_kwargs.
                    Please confirm that endpoint_url is what you intended.
WARNING! task is not default parameter.
                    task was transferred to model_kwargs.
                    Please confirm that task is what you intended.
WARNING! huggingfacehub_api_token is not default parameter.
                    huggingfacehub_api_token was transferred to model_kwargs.
                    Please confirm that huggingfacehub_api_token is what you intended.


last message must be a HumanMessage
System prompt not supported


  0%|          | 0/100 [00:00<?, ?it/s]

not found


> Entering new AgentExecutor chain...


 68%|██████▊   | 68/100 [01:22<00:38,  1.22s/it]


CancelledError: 

# Evaluate with LLM-as-a-judge

In [29]:
eval_chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
eval_model_name = "GPT4"

In [33]:
for file in tqdm(glob.glob("output/*.json")):
    evaluate_answers(
        file,
        eval_chat_model,
        eval_model_name,
        EVALUATION_PROMPT_TEMPLATE,
    )
print("Evaluation is complete!")

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/87 [00:00<?, ?it/s]

  0%|          | 0/87 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/87 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Evaluation is complete!


## Visualize results

In [1]:
result_df = pd.concat([pd.read_json(f) for f in glob.glob("output/*.json")])
result_df["no_prediction"] = result_df["prediction"].apply(lambda x: True if x is None else False)
result_df["has_agent_error"] = result_df["agent_error"].apply(
    lambda x: True if isinstance(x, str) else False
)


def interpret_result(x):
    try:
        return int(x) - 1
    except:
        return 0


result_df["eval_score_Mixtral"] = result_df["eval_score_Mixtral"].apply(interpret_result)

# Override results with human evaluation if there is one
result_df["eval_score_GPT4"] = result_df.apply(
    lambda row: row["eval_score_human"]
    if not np.isnan(row["eval_score_human"])
    else row["eval_score_GPT4"],
    axis=1,
)
result_df["eval_score_GPT4"] = result_df["eval_score_GPT4"].apply(interpret_result)

result_df["task"] = result_df["task"].apply(lambda x: ("HotpotQA" if "HotpotQA" in x else x))
result_df.groupby(["agent_name", "task"]).agg(
    {
        "parsing_error": "sum",
        "iteration_limit_exceeded": "sum",
        "no_prediction": "sum",
        "has_agent_error": "sum",
    }
)

NameError: name 'pd' is not defined

In [40]:
table_result = result_df.groupby(["agent_name", "task"], sort=False)[["eval_score_GPT4"]].mean()
table_result = table_result / 4 * 100  # set results in percentage
display(table_result.unstack())

eval_score_GPT4                      
task                                               GSM8K   HotpotQA       GAIA
agent_name                                                                    
OpenHermes-2.5-Mistral-7B                          46.25  65.833333   0.000000
Llama-2-70b-chat                                   17.50  49.583333   0.000000
Mixtral-8x7B-Instruct-v0.1_changedprompt           72.50  77.083333  16.250000
Zephyr-7b-beta                                     46.25  35.000000   0.000000
GPT4                                               95.00  80.833333  40.000000
Mixtral-8x7B-Instruct-v0.1                         67.50  69.166667  21.428571
GPT3.5                                             62.50  74.583333  16.250000

In [41]:
# Assert that all agents were tried on all questions
# np.testing.assert_array_equal(
#     result_df["agent_name"].value_counts().values,
#     [100] * len(result_df["agent_name"].value_counts()),
# )

table_result = table_result.reset_index()
table_result = table_result.sort_values("agent_name")
# Plot results
fig = px.bar(
    table_result,
    x="agent_name",
    y="eval_score_GPT4",
    color="task",
    title=f"Average Evaluation Score (LLM-as-a-judge)",
    labels={
        "agent_name": "Agent",
        "task": "Task",
        "score": "Performance",
        "eval_score_GPT4": "Score",
    },
)
fig.update_layout(
    width=1000,
    height=600,
    barmode="group",
    yaxis_range=[0, 100],
)
fig.update_traces(texttemplate="%{y:.0f}", textposition="outside")
fig.layout.yaxis.ticksuffix = "%"
fig.show()

### Study intermediate steps

In [ ]:
try:
    result_df["tools_used"] = result_df["intermediate_steps"].apply(
        lambda row: ([step["tool"] for step in row] if row is not None else None)
    )
except:
    pass
result_df["has_answer"] = result_df["prediction"].apply(lambda row: (row is not None))
result_df_answers_only = result_df.loc[result_df["has_answer"] == True]
result_df_answers_only["correct_answer"] = (
    result_df_answers_only[f"eval_score_{eval_model_name}"] >= 4
)
result_df_answers_only["number_of_steps"] = result_df_answers_only["tools_used"].apply(
    lambda x: len(x) + 1
)
aggregated_resuts = result_df_answers_only.groupby(["agent_name", "task", "correct_answer"]).agg(
    {"number_of_steps": "mean"}
)

/tmp/ipykernel_2321/2687269850.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_2321/2687269850.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
px.bar(
    aggregated_resuts.reset_index(),
    x="task",
    y="number_of_steps",
    facet_col="agent_name",
    color="correct_answer",
    title="Mean Number of Steps to Solve a Task",
    barmode="group",
    labels={
        "agent_name": "Agent",
        "number_of_steps": "Mean Number of Steps",
        "correct_answer": "Answer correctness",
        "task": "Task",
    },
    width=1500,
)

Here we can see that on average, __tasks that failed have a higher number of steps taken than successful tasks__: taking many steps seems to be an indication that the agent is trying wrong directions to solve the problem.

### Display Prometheus vs GPT4

In [ ]:
res = pd.read_json("output/GPT4.json")
res = res.loc[~res["eval_score_human"].isnull()]
res = res[["eval_score_human", "eval_score_GPT4", "eval_score_Prometheus-13B-v1.0"]].reset_index(
    drop=True
)

In [ ]:
fig = px.bar(
    res,
)
fig.update_layout(
    width=1100,
    height=300,
    barmode="group",
    yaxis_range=[0.5, 5.5],
    yaxis_title="Score",
    yaxis=dict(
        tickmode="array",
        tickvals=[1, 3, 5],
    ),
)
fig.show()

In [ ]:
print("Proportion of cases matching human eval:")
print(
    (res["eval_score_human"] == res["eval_score_Prometheus-13B-v1.0"]).mean().round(3),
    "for Prometheus",
)
print((res["eval_score_human"] == res["eval_score_GPT4"]).mean().round(3), "for GPT4")

Proportion of cases matching human eval:
0.333 for Prometheus
0.967 for GPT4


We could not use Prometheus-13B for this evaluation given its high rate of error.

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, load_tools
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import (
    ReActJsonSingleInputOutputParser,
)
from langchain.tools.render import render_text_description
from langchain_community.utilities import SerpAPIWrapper

# setup tools

# setup ReAct style prompt
prompt = hub.pull("hwchase17/react-json")

In [ ]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).\n\nThe only values that should be in the "action" field are: {tool_names}\n\nThe $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nALWAYS use the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction:\n```\n$JSON_BLOB\n```\nObservation: the result of the action\